# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd

# Functions

# Main

In [1]:
from base.systems import HardDiskSystem
box_size = 10
n_particles = 5
particle_radius = 0.1
system = HardDiskSystem(box_size = box_size, n_particles=n_particles,particle_radius=particle_radius)
print(system)

HardDiskSystem(box_size=10, particle_radius=0.1, n_particles=5, seed=42, max_velocity=1, periodict_boundary=False, rng=Generator(PCG64) at 0x7F46CAAAD7E0, positions=array([[7.73956049, 4.3887844 ],
       [8.5859792 , 6.97368029],
       [0.94177348, 9.75622352],
       [7.61139702, 7.86064305],
       [1.28113633, 4.50385938]]), velocities=array([[-0.25840395,  0.85352998],
       [ 0.28773024,  0.64552323],
       [-0.1131716 , -0.54552256],
       [ 0.10916957, -0.87236549],
       [ 0.65526234,  0.2633288 ]]))
